In [1]:
import torch
import torch.nn as nn

In [6]:
class Bi_LSTM(nn.Module):
    def __init__(self, model_parameter):
        super(Bi_LSTM, self).__init__()
        self.input_size = model_parameter.input_size
        self.hidden_size = model_parameter.hidden_size
        self.output_size = model_parameter.output_size
        self.future_step = model_parameter.future_step
        self.batch_size = model_parameter.batch_size
        self.lag_window = model_parameter.lag_window

        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(input_size= 2 * self.hidden_size, hidden_size=self.hidden_size, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(2 * self.hidden_size * self.lag_window, self.future_step * self.output_size)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        lstm, (_,_) = self.lstm(x)
        #lstm, (_,_) = self.lstm2(lstm)
        lstm = self.flatten(lstm)
        final = self.linear(lstm)
        final = final.view(self.batch_size, self.future_step, -1)
        return self.relu(final)